<a href="https://colab.research.google.com/github/vihaveman/Breaking_Barriers/blob/Violetta/Prediction_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
!pip install pandas
!pip install scikit-learn
!pip install tensorflow
!pip install tensorflow

ERROR: Operation cancelled by user


In [2]:
import pandas as pd
from google.colab import files

# Upload the file to Colab environment
uploaded = files.upload()

Saving NN_Pred.csv to NN_Pred.csv


In [49]:
import io

# Load the data into a DataFrame
data = pd.read_csv(io.BytesIO(uploaded['NN_Pred.csv']))

# Drop unnecessary columns
data1 = data.drop(columns=['Country Name', 'Country Code'])

# Check for missing values and handle them (e.g., fill with mean or median)
data1.fillna(data.mean(), inplace=True)

# Normalize the features (optional but recommended for neural networks)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[data1.columns[:-1]] = scaler.fit_transform(data1[data1.columns[:-1]])


<ipython-input-49-2a01d43fe920>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data1.fillna(data.mean(), inplace=True)


In [50]:
data.head(5)

,Country Name,Country Code,Year,ATM Usage,GDP,High Tech Export ($M),High Tech Import,Internet Subscription Per 100,Internet Usage Per 100,IP Income($M),Labour Force (M),Med_High Tech Manufacturing,Mobile Sub Per 100,Personal Remittance ($M),Poverty Count,Stock Traded $B,Tech Cooperation Grant($M),Unemployment Rate,Med High Tech Export
0,Afghanistan,AFG,-1.658312,0.0,-0.236269,-1.415184e-16,0.326169,0.00000,-1.182784,-0.257097,-0.318100,-0.599262,-1.449760,-0.267337,0.000000,-0.226678,-0.137695,0.026478,0.00
1,Africa Eastern and Southern,AFE,-1.658312,0.0,-0.255547,-1.415184e-16,-0.046757,0.00000,-1.172630,-0.252306,0.053087,0.000000,-1.402278,-0.261782,0.000000,-0.177421,-0.147597,-0.170244,NaN
2,Africa Western and Central,AFW,-1.658312,0.0,-0.215062,-1.415184e-16,0.699095,0.00000,-1.193265,-0.261889,-0.092168,-0.956580,-1.439814,-0.272891,0.000000,-0.275811,-0.124607,-0.616145,0.00
3,Albania,ALB,-1.658312,0.0,1.070833,-1.415184e-16,-0.410621,0.00000,-1.194247,-0.256692,-0.326101,-1.013363,-1.432119,-0.303066,0.629298,-0.374078,-0.102480,2.418982,6.56
4,Algeria,DZA,-1.658312,0.0,0.052913,-1.415184e-16,-0.547941,-0.75116,-1.181801,-0.247354,-0.307789,-0.759224,-1.444505,-0.296166,0.391508,-0.348711,-0.083508,3.816639,2.22


In [40]:
#Split the data into features and target
X = data1.drop(columns=['Poverty Count',])
y = data1['Poverty Count']


In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.regularizers import l2
import numpy as np


# Build the neural network model with L2 regularization in the hidden layers
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='linear', kernel_regularizer=l2(0.01)))  # Linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model and save the training history
history = model.fit(X_train, y_train, epochs=10, batch_size=25, validation_split=0.2, verbose=1)

# Evaluate the model on training and testing data
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)



Epoch 1/10
157/157 [==============================] - 1s 4ms/step - loss: 2.2331 - val_loss: 1.2605
Epoch 2/10
157/157 [==============================] - 0s 3ms/step - loss: 1.1651 - val_loss: 1.1311
Epoch 3/10
157/157 [==============================] - 0s 3ms/step - loss: 1.0972 - val_loss: 1.0752
Epoch 4/10
157/157 [==============================] - 0s 3ms/step - loss: 1.0467 - val_loss: 1.0642
Epoch 5/10
157/157 [==============================] - 0s 3ms/step - loss: 1.0211 - val_loss: 1.0211
Epoch 6/10
157/157 [==============================] - 0s 3ms/step - loss: 0.9835 - val_loss: 0.9768
Epoch 7/10
157/157 [==============================] - 0s 3ms/step - loss: 0.9610 - val_loss: 0.9603
Epoch 8/10
157/157 [==============================] - 0s 3ms/step - loss: 0.9586 - val_loss: 0.9421
Epoch 9/10
157/157 [==============================] - 1s 4ms/step - loss: 0.9251 - val_loss: 0.9314
Epoch 10/10
39/39 [==============================] - 0s 2ms/step


In [14]:
# Calculate Mean Squared Error (MSE)
train_mse = np.mean((train_pred - y_train.values.reshape(-1, 1)) ** 2)
test_mse = np.mean((test_pred - y_test.values.reshape(-1, 1)) ** 2)

print("Training MSE:", train_mse)
print("Testing MSE:", test_mse)


Training MSE: 0.7511499305111764
Testing MSE: 0.8445012442685428


In [15]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                1088      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


In [17]:
print(data.shape)

(6118, 17)


In [18]:
model.save("trained_model.h5")
# Download the trained model file to your local machine
files.download("trained_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
import shutil

# Define the source path of the model file
source_path = "trained_model.h5"

# Define the destination path where you want to download the model file
destination_path = "/content/trained_model.h5"

# Use shutil to move the model file to the destination directory
shutil.move(source_path, destination_path)

# The trained model file has now been downloaded to the specified destination directory

'/content/trained_model.h5'

In [20]:
# Load the trained model
from tensorflow.keras.models import load_model

model = load_model('/content/trained_model.h5')


In [27]:
# Generate the 'Year' range for the next five years (2024 to 2028)
next_five_years = range(2024, 2029)

# Flatten the predictions array
flat_predictions = predictions.flatten()

# Check the lengths of both arrays
print(len(next_five_years))
print(len(flat_predictions))

# Trim or pad the predictions array to match the length of the 'next_five_years' range
if len(flat_predictions) < len(next_five_years):
    # Pad the predictions array with NaN values
    flat_predictions = np.pad(flat_predictions, (0, len(next_five_years) - len(flat_predictions)), mode='constant', constant_values=np.nan)
elif len(flat_predictions) > len(next_five_years):
    # Trim the predictions array to match the length of 'next_five_years' range
    flat_predictions = flat_predictions[:len(next_five_years)]

# Create a new DataFrame with the predictions and the corresponding years
predicted_df = pd.DataFrame({'Year': next_five_years, 'Predicted_Poverty_Count': flat_predictions})

# Display the head of the new DataFrame
print(predicted_df.head()



5
6118
   Year  Predicted_Poverty_Count
0  2024                 0.319257
1  2025                 0.784652
2  2026                 0.278881
3  2027                 0.793579
4  2028                 0.770337


In [25]:
model.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                1088      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Step 2: Predict poverty count for each country for the next five years
predicted_dfs = []
next_five_years = range(2024, 2029)  # Adjust this range based on the number of predictions (e.g., 5 years)
for country, country_data in grouped_data:
    # Get features (X) for the current country
    X_country = country_data.drop(columns=['Country Name', 'Poverty Count'])  # Keep all features except 'Country Name' and 'Poverty Count'

    # Use the trained model to predict poverty count for the next five years
    country_predictions = model.predict(X_country)

    # Flatten the predictions array
    flat_predictions = country_predictions.flatten()

    # Trim or pad the predictions array to match the length of the 'next_five_years' range
    if len(flat_predictions) < len(next_five_years):
        # Pad the predictions array with NaN values
        flat_predictions = np.pad(flat_predictions, (0, len(next_five_years) - len(flat_predictions)), mode='constant', constant_values=np.nan)
    elif len(flat_predictions) > len(next_five_years):
        # Trim the predictions array to match the length of 'next_five_years' range
        flat_predictions = flat_predictions[:len(next_five_years)]

    # Create a DataFrame with the predictions and the corresponding years for the current country
    predicted_df = pd.DataFrame({'Year': next_five_years, 'Country Name': country, 'Predicted_Poverty_Count': flat_predictions})

    # Append the DataFrame for the current country to the list
    predicted_dfs.append(predicted_df)

# Step 3: Concatenate DataFrames for all countries into one DataFrame
predicted_df = pd.concat(predicted_dfs, ignore_index=True)



1/1 [==============================] - 0s 29ms/step


,Year,Country Name,Predicted_Poverty_Count
0,2024,Afghanistan,0.319257
1,2025,Afghanistan,0.290222
2,2026,Afghanistan,0.259798
3,2027,Afghanistan,0.298528
4,2028,Afghanistan,0.310322
5,2024,Africa Eastern and Southern,NaN
6,2025,Africa Eastern and Southern,NaN
7,2026,Africa Eastern and Southern,NaN
8,2027,Africa Eastern and Southern,NaN
9,2028,Africa Eastern and Southern,NaN


In [69]:
# Display the head of the new DataFrame
predicted_df.head()

,Year,Country Name,Predicted_Poverty_Count
0,2024,Afghanistan,0.319257
5,2024,Africa Eastern and Southern,NaN
10,2024,Africa Western and Central,0.278881
15,2024,Albania,0.793579
20,2024,Algeria,0.770337


In [70]:
# Sort the DataFrame by 'Year' first and then by 'Country Name'
predicted_df = predicted_df.sort_values(by=['Year', 'Country Name'])

# Display the sorted DataFrame
predicted_df.head()


,Year,Country Name,Predicted_Poverty_Count
0,2024,Afghanistan,0.319257
5,2024,Africa Eastern and Southern,NaN
10,2024,Africa Western and Central,0.278881
15,2024,Albania,0.793579
20,2024,Algeria,0.770337


In [72]:
# Reorder the columns in the DataFrame
predicted_df[['Country Name', 'Year'] + [col for col in predicted_df.columns if col not in ['Country Name', 'Year']]]

# Display the reordered DataFrame
predicted_df.head()

,Year,Country Name,Predicted_Poverty_Count
0,2024,Afghanistan,0.319257
5,2024,Africa Eastern and Southern,NaN
10,2024,Africa Western and Central,0.278881
15,2024,Albania,0.793579
20,2024,Algeria,0.770337


In [78]:
import os

# Reorder the columns in the DataFrame
predicted_df_reordered = predicted_df[['Country Name', 'Year'] + [col for col in predicted_df.columns if col not in ['Country Name', 'Year']]]

# Define the directory path for the CSV file
directory_path = '/content/drive/My Drive/Colab Notebooks/resources/'

# Create the 'resources' folder if it doesn't exist
os.makedirs(directory_path, exist_ok=True)

# Define the file path for the CSV file
file_path = os.path.join(directory_path, 'Predicted_Inflation.csv')

# Export the DataFrame to a CSV file
predicted_df_reordered.to_csv(file_path, index=False)
